# MoE Kernel Benchmark Analysis

This notebook analyzes benchmark results

## Hypotheses to Test:
1. **Does the activation affect the time and kernel selection? (why?)**

In [4]:
# Imports (if not already run)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Libraries imported successfully!")

Libraries imported successfully!


In [6]:
# Activation Comparison Table for Jupyter Notebook
# Copy these cells into moe_kernel_analysis.ipynb

# ============================================================================
# Cell 1: Load Data
# ============================================================================

# Load activation comparison results
act_df = pd.read_csv('results/results_activation_comparison.csv')

# Filter valid results
valid_df = act_df[act_df['error'] != 'failed'].copy()
valid_df['error_pct'] = valid_df['error'].str.rstrip('%').astype(float)
valid_df = valid_df[valid_df['error_pct'] < 50.0]

print(f"Loaded {len(valid_df)} valid kernel results")
print(f"  Silu: {len(valid_df[valid_df['act_type'] == 'ActivationType.Silu'])}")
print(f"  Gelu: {len(valid_df[valid_df['act_type'] == 'ActivationType.Gelu'])}")

# ============================================================================
# Cell 2: Create Comparison Table
# ============================================================================

print("\nCreating Silu vs Gelu comparison table...")

# Normalize kernel names (remove activation)
def normalize_kernel_name(kernel_name):
    base = kernel_name
    if '_silu_' in base.lower():
        base = base.replace('_silu_', '_ACT_').replace('_Silu_', '_ACT_')
    elif '_gelu_' in base.lower():
        base = base.replace('_gelu_', '_ACT_').replace('_Gelu_', '_ACT_')
    return base

valid_df['kernel_base_name'] = valid_df['kernel_name'].apply(normalize_kernel_name)


# Define matching columns
match_cols = [
    'token', 'model_dim', 'inter_dim', 'expert', 'topk',
    'dtype', 'q_dtype_a', 'q_dtype_w', 'q_type',
    'use_g1u1', 'doweight_stage1',
    'stage', 'block_m', 'kernel_type', 'tile_m', 'tile_n',
    'kernel_base_name'
]

# Separate Silu and Gelu
silu_df = valid_df[valid_df['act_type'] == 'ActivationType.Silu'].copy()
gelu_df = valid_df[valid_df['act_type'] == 'ActivationType.Gelu'].copy()

# Merge on matching columns
comparison = silu_df.merge(
    gelu_df,
    on=match_cols,
    how='inner',
    suffixes=('_silu', '_gelu')
)

# Calculate differences
comparison['time_diff_us'] = comparison['time_us_gelu'] - comparison['time_us_silu']
comparison['time_diff_pct'] = (comparison['time_diff_us'] / comparison['time_us_silu']) * 100
comparison['faster'] = comparison.apply(
    lambda row: 'Silu' if row['time_us_silu'] < row['time_us_gelu'] else 'Gelu',
    axis=1
)

# Select display columns
display_cols = [
    'config_idx_silu', 'token', 'model_dim', 'expert', 'topk', 
    'stage', 'kernel_type', 'block_m',  # Added kernel_type and block_m
    'kernel_name_silu', 'kernel_name_gelu',
    'time_us_silu', 'time_us_gelu', 
    'time_diff_pct', 'faster',
    'error_silu', 'error_gelu'
]

result = comparison[display_cols].copy()
result = result.rename(columns={'config_idx_silu': 'config_idx'})

# Sort by absolute difference
result = result.sort_values('time_diff_pct', key=abs, ascending=False)

print(f"Matched kernel pairs: {len(result)}")
print(f"Silu faster: {(result['faster'] == 'Silu').sum()} ({(result['faster'] == 'Silu').sum()/len(result)*100:.1f}%)")
print(f"Gelu faster: {(result['faster'] == 'Gelu').sum()} ({(result['faster'] == 'Gelu').sum()/len(result)*100:.1f}%)")
print(f"Average difference: {result['time_diff_pct'].abs().mean():.2f}%")

# ============================================================================
# Cell 3: Display Table
# ============================================================================

print("\nComparison Table (sorted by largest performance difference):")
print("="*80)
display(result.head(20))

print("\n**Table Columns:**")
print("- time_us_silu: Kernel execution time with Silu activation")
print("- time_us_gelu: Kernel execution time with Gelu activation")
print("- time_diff_pct: Percentage difference (Gelu relative to Silu)")
print("  - Negative = Silu faster")
print("  - Positive = Gelu faster")
print("- faster: Which activation is faster for this kernel")

# ============================================================================
# Cell 4: Summary Statistics
# ============================================================================

print("\nSummary Statistics:")
print("="*80)

print(f"\nOverall:")
print(f"  Matched pairs: {len(result)}")
print(f"  Average |difference|: {result['time_diff_pct'].abs().mean():.2f}%")
print(f"  Median |difference|: {result['time_diff_pct'].abs().median():.2f}%")
print(f"  Max |difference|: {result['time_diff_pct'].abs().max():.2f}%")

print(f"\nBy Stage:")
for stage in result['stage'].unique():
    stage_data = result[result['stage'] == stage]
    print(f"  {stage}: {len(stage_data)} kernels, avg diff = {stage_data['time_diff_pct'].abs().mean():.2f}%")

if 'kernel_type' in result.columns:
    print(f"\nBy Kernel Type:")
    for ktype in result['kernel_type'].unique():
        ktype_data = result[result['kernel_type'] == ktype]
        silu_wins = (ktype_data['faster'] == 'Silu').sum()
        print(f"  {ktype}: Silu wins {silu_wins}/{len(ktype_data)} ({silu_wins/len(ktype_data)*100:.1f}%)")

# ============================================================================
# Cell 5: Interactive Visualization (2x2 Grid with Plotly)
# ============================================================================

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create plot groups (stage1-asm, stage1-ck, stage2, asm_1stage)
plot_groups = []

stage1_data = result[result['stage'] == 'stage1']
for ktype in sorted(stage1_data['kernel_type'].unique()):
    data = stage1_data[stage1_data['kernel_type'] == ktype]
    plot_groups.append((f'Stage1-{ktype.upper()}', data))

for stage in ['stage2', 'asm_1stage']:
    stage_data = result[result['stage'] == stage]
    if len(stage_data) > 0:
        plot_groups.append((stage, stage_data))

# Create 2x2 subplot grid
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[f'{label} ({len(data)} kernels)' for label, data in plot_groups],
    horizontal_spacing=0.12,
    vertical_spacing=0.15
)

for idx, (label, data) in enumerate(plot_groups):
    row = (idx // 2) + 1
    col = (idx % 2) + 1
    
    # Create hover text with kernel details
    hover_text = []
    for _, r in data.iterrows():
        block_m_info = f"block_m={r['block_m']}<br>" if 'block_m' in data.columns else ""
        text = (f"<b>Config #{r['config_idx']}</b><br>"
                f"Token={r['token']}, Model={r['model_dim']}, Expert={r['expert']}, TopK={r['topk']}<br>"
                f"{block_m_info}"
                f"<br><b>Silu Kernel:</b><br>{r['kernel_name_silu'][:80]}<br>"
                f"Time: {r['time_us_silu']:.2f} us (err={r['error_silu']})<br>"
                f"<br><b>Gelu Kernel:</b><br>{r['kernel_name_gelu'][:80]}<br>"
                f"Time: {r['time_us_gelu']:.2f} us (err={r['error_gelu']})<br>"
                f"<br><b>Performance:</b> {r['time_diff_pct']:.2f}% ({r['faster']} faster)")
        hover_text.append(text)
    
    # Add scatter trace
    fig.add_trace(
        go.Scatter(
            x=data['time_us_silu'],
            y=data['time_us_gelu'],
            mode='markers',
            marker=dict(
                size=8,
                color=data['time_diff_pct'],
                colorscale='RdYlGn_r',
                cmin=-10,
                cmax=10,
                showscale=False,  # Remove colorbar
                line=dict(width=1, color='DarkSlateGray')
            ),
            hovertext=hover_text,
            hoverinfo='text',
            name=label,
            showlegend=False
        ),
        row=row, col=col
    )
    
    # Add diagonal line (equal performance)
    if len(data) > 0:
        max_val = max(data['time_us_silu'].max(), data['time_us_gelu'].max())
        fig.add_trace(
            go.Scatter(
                x=[0, max_val],
                y=[0, max_val],
                mode='lines',
                line=dict(color='red', dash='dash', width=2),
                name='Equal Performance',
                showlegend=(idx == 0),
                hoverinfo='skip'
            ),
            row=row, col=col
        )
    
    # Update axes
    fig.update_xaxes(title_text='Silu Time (us)', row=row, col=col)
    fig.update_yaxes(title_text='Gelu Time (us)', row=row, col=col)

# Update layout
fig.update_layout(
    height=900,
    width=1400,
    title_text='Interactive Kernel Performance: Silu vs Gelu<br><sub>Hover over points for details</sub>',
    title_font_size=18,
    showlegend=True,
    hovermode='closest'
)

fig.show()

print("\n✅ Interactive Features:")
print("  - Hover over any point to see full kernel details")
print("  - Zoom: Click and drag to select area")
print("  - Pan: Hold shift and drag")
print("  - Reset: Double-click")
print("\n📊 Color Legend:")
print("  - Red (above diagonal) = Gelu is SLOWER")
print("  - Green (below diagonal) = Gelu is FASTER")
print("  - White (on diagonal) = Equal performance")


Loaded 1444 valid kernel results
  Silu: 722
  Gelu: 722

Creating Silu vs Gelu comparison table...
Matched kernel pairs: 722
Silu faster: 397 (55.0%)
Gelu faster: 325 (45.0%)
Average difference: 0.69%

Comparison Table (sorted by largest performance difference):


,config_idx,token,model_dim,expert,topk,stage,kernel_type,block_m,kernel_name_silu,kernel_name_gelu,time_us_silu,time_us_gelu,time_diff_pct,faster,error_silu,error_gelu
383,26,16,5120,128,1,stage1,asm,128,_ZN5aiter53fmoe_stage1_bf16_pertokenFp8_doweig...,_ZN5aiter53fmoe_stage1_bf16_pertokenFp8_doweig...,54.4555,60.9869,11.994013,Silu,0.00%,0.00%
382,26,16,5120,128,1,stage1,asm,64,_ZN5aiter57fmoe_stage1_bf16_pertokenFp8_doweig...,_ZN5aiter57fmoe_stage1_bf16_pertokenFp8_doweig...,54.3252,59.0300,8.660438,Silu,0.00%,0.00%
390,26,16,5120,128,1,stage1,asm,16,_ZN5aiter57fmoe_stage1_bf16_pertokenFp8_doweig...,_ZN5aiter57fmoe_stage1_bf16_pertokenFp8_doweig...,63.8575,58.9068,-7.752731,Gelu,0.00%,0.00%
123,8,32,7168,256,8,stage1,asm,32,_ZN5aiter53fmoe_stage1_bf16_pertokenFp8_doweig...,_ZN5aiter53fmoe_stage1_bf16_pertokenFp8_doweig...,299.8873,319.3323,6.484103,Silu,0.00%,0.00%
411,26,16,5120,128,1,asm_1stage,asm,32,_ZN5aiter50fmoe_bf16_pertokenFp8_g1u1_vs_tkw1_...,_ZN5aiter50fmoe_bf16_pertokenFp8_g1u1_vs_tkw1_...,117.0026,124.2754,6.215930,Silu,0.01%,0.08%
341,24,128,7168,33,10,stage1,asm,32,_ZN5aiter47fmoe_stage1_bf16_pertokenFp8_g1u1_3...,_ZN5aiter47fmoe_stage1_bf16_pertokenFp8_g1u1_3...,269.8343,285.9037,5.955284,Silu,0.00%,0.00%
82,6,512,7168,256,8,stage2,ck,32,moe_ck2stages_gemm2_256x32x64x256_1x4_MulABSca...,moe_ck2stages_gemm2_256x32x64x256_1x4_MulABSca...,143.5682,151.8052,5.737343,Silu,1.29%,1.62%
380,26,16,5120,128,1,stage1,asm,64,_ZN5aiter56fmoe_stage1_bf16_pertokenFp8_doweig...,_ZN5aiter56fmoe_stage1_bf16_pertokenFp8_doweig...,48.6242,51.2520,5.404305,Silu,0.00%,0.00%
521,40,32,7168,256,8,stage1,asm,64,_ZN5aiter48fmoe_stage1_bf16_pertokenFp8_g1u1_6...,_ZN5aiter48fmoe_stage1_bf16_pertokenFp8_g1u1_6...,156.8232,165.1923,5.336647,Silu,0.00%,0.00%
475,36,16,7168,256,8,stage2,ck,64,moe_ck2stages_gemm2_256x64x64x128_1x4_TypeCast...,moe_ck2stages_gemm2_256x64x64x128_1x4_TypeCast...,109.3758,114.5627,4.742274,Silu,1.41%,1.67%



**Table Columns:**
- time_us_silu: Kernel execution time with Silu activation
- time_us_gelu: Kernel execution time with Gelu activation
- time_diff_pct: Percentage difference (Gelu relative to Silu)
  - Negative = Silu faster
  - Positive = Gelu faster
- faster: Which activation is faster for this kernel

Summary Statistics:

Overall:
  Matched pairs: 722
  Average |difference|: 0.69%
  Median |difference|: 0.28%
  Max |difference|: 11.99%

By Stage:
  stage1: 516 kernels, avg diff = 0.73%
  asm_1stage: 49 kernels, avg diff = 0.56%
  stage2: 157 kernels, avg diff = 0.61%

By Kernel Type:
  asm: Silu wins 272/417 (65.2%)
  ck: Silu wins 125/305 (41.0%)



✅ Interactive Features:
  - Hover over any point to see full kernel details
  - Zoom: Click and drag to select area
  - Pan: Hold shift and drag
  - Reset: Double-click

📊 Color Legend:
  - Red (above diagonal) = Gelu is SLOWER
  - Green (below diagonal) = Gelu is FASTER
  - White (on diagonal) = Equal performance


In [18]:
#!/usr/bin/env python3
"""
Interactive visualization of slowdown factors across configurations.
"""

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load best kernels with reference
print("Loading best_kernels_with_reference.csv...")
df = pd.read_csv('results/best_kernels_with_reference.csv')

print(f"Loaded {len(df)} configurations")

# Sort by config_idx
df = df.sort_values('config_idx')

# Create hover text with details
hover_text = []
for _, row in df.iterrows():
    text = (f"<b>Config #{row['config_idx']}</b><br>"
            f"Token={row['token']}, Model={row['model_dim']}, Expert={row['expert']}, TopK={row['topk']}<br>"
            f"Approach: {row['approach']}<br>"
            f"<br><b>Best Kernel:</b><br>"
            f"{row['stage1_kernel_name'][:60]}<br>"
            f"<br><b>Performance:</b><br>"
            f"Actual: {row['total_time_us']:.2f} us<br>"
            f"Theoretical: {row['theoretical_time_us']:.2f} us<br>"
            f"Slowdown: {row['slowdown_factor']:.2f}x<br>")
    hover_text.append(text)

# Create interactive plot
fig = go.Figure()

# Add scatter trace
fig.add_trace(go.Scatter(
    x=df['config_idx'],
    y=df['slowdown_factor'],
    mode='markers+lines',
    marker=dict(
        size=10,
        color=df['slowdown_factor'],
        colorscale='RdYlGn_r',
        showscale=True,
        colorbar=dict(title='Slowdown<br>Factor'),
        line=dict(width=1, color='black')
    ),
    line=dict(width=1, color='lightblue'),
    hovertext=hover_text,
    hoverinfo='text',
    name='Slowdown Factor'
))

# Add reference line at 1.0 (theoretical performance)
# fig.add_hline(y=1.0, line_dash="dash", line_color="green", 
#               annotation_text="Theoretical Optimum (1.0x)", 
#               annotation_position="right")


# Update layout
fig.update_layout(
    title='Slowdown Factor Across Configurations<br><sub>Hover for details, zoom/pan to explore</sub>',
    xaxis_title='Configuration Index',
    yaxis_title='Slowdown Factor (Actual / Theoretical)',
    height=700,
    width=1800,
    hovermode='closest',
    showlegend=True
)

# Update axes
fig.update_xaxes(showgrid=True, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridcolor='lightgray', type='log')

fig.show()



Loading best_kernels_with_reference.csv...
Loaded 679 configurations


In [ ]:
#!/usr/bin/env python3
"""
Multi-view interactive visualization of kernel performance.

Creates separate plots with different perspectives:
1. Config Index vs Slowdown (colored by approach)
2. Token vs Slowdown (colored by expert count)
3. Token vs Expert (colored by slowdown)
4. Expert vs Model Dim (colored by slowdown)
"""

import pandas as pd
import plotly.graph_objects as go

# Load best kernels with reference
print("Loading best_kernels_with_reference.csv...")
df = pd.read_csv('results/best_kernels_with_reference.csv')

print(f"Loaded {len(df)} configurations")

# Sort by config_idx
df = df.sort_values('config_idx')

# Common hover text generator
def create_hover(row):
    return (f"<b>Config #{row['config_idx']}</b><br>"
            f"T={row['token']}, M={row['model_dim']}, I={row['inter_dim']}<br>"
            f"E={row['expert']}, K={row['topk']}<br>"
            f"Quant: {row['q_type']}<br>"
            f"Approach: {row['approach']}<br>"
            f"Activation: {row['act_type']}<br>"
            f"block_m: {row['block_m']}<br>"
            f"Kernel: {row['stage1_kernel_name'][:50]}<br>"
            f"Actual: {row['total_time_us']:.2f} us<br>"
            f"Theoretical: {row['theoretical_time_us']:.2f} us<br>"
            f"Slowdown: {row['slowdown_factor']:.2f}x")

# =============================================================================
# Figure 1: Config Index vs Slowdown (with dropdown for color selection)
# =============================================================================
print("\nCreating Figure 1: Config Index vs Slowdown (with color options)...")

fig1 = go.Figure()

hover = [create_hover(row) for _, row in df.iterrows()]

# Create traces for each color_by option
color_by_options = {
    'approach': 'Approach (2-stage/1-stage)',
    'q_type': 'Quantization Type',
    'topk': 'TopK',
    'token': 'Token Count',
    'model_dim': 'Model Dimension',
    'inter_dim': 'Intermediate Dimension',
    'expert': 'Expert Count',
    'use_g1u1': 'use_g1u1',
    'doweight_stage1': 'doweight_stage1',
    'block_m': 'Block M',
    'act_type': 'Activation Type'
}

# For each color option, create data (all initially invisible except first)
for idx, (col_name, col_label) in enumerate(color_by_options.items()):
    if col_name in ['approach', 'q_type', 'act_type', 'use_g1u1', 'doweight_stage1']:
        # Categorical - separate traces per category
        for cat in df[col_name].unique():
            cat_data = df[df[col_name] == cat]
            cat_hover = [create_hover(row) for _, row in cat_data.iterrows()]
            
            fig1.add_trace(go.Scatter(
                x=cat_data['config_idx'],
                y=cat_data['slowdown_factor'],
                mode='markers',
                marker=dict(size=8),
                hovertext=cat_hover,
                hoverinfo='text',
                name=str(cat),
                visible=(col_name == 'approach')  # Only 'approach' visible initially
            ))
    else:
        # Numerical - use colorscale
        fig1.add_trace(go.Scatter(
            x=df['config_idx'],
            y=df['slowdown_factor'],
            mode='markers',
            marker=dict(
                size=8,
                color=df[col_name],
                colorscale='Viridis',
                colorbar=dict(title=col_label),
                line=dict(width=1, color='black')
            ),
            hovertext=hover,
            hoverinfo='text',
            showlegend=False,
            visible=False
        ))

# Create dropdown menu
buttons = []
trace_idx = 0

for col_name, col_label in color_by_options.items():
    # Count traces for this option
    if col_name in ['approach', 'q_type', 'act_type', 'use_g1u1', 'doweight_stage1']:
        n_cats = df[col_name].nunique()
        visibility = [False] * len(fig1.data)
        for i in range(trace_idx, trace_idx + n_cats):
            visibility[i] = True
        trace_idx += n_cats
    else:
        visibility = [False] * len(fig1.data)
        visibility[trace_idx] = True
        trace_idx += 1
    
    buttons.append(dict(
        label=col_label,
        method='update',
        args=[{'visible': visibility}]
    ))

fig1.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        direction='down',
        x=0.15,
        xanchor='left',
        y=1.15,
        yanchor='top'
    )],
    title='Config Index vs Slowdown Factor<br><sub>Select color-by from dropdown</sub>',
    xaxis_title='Config Index',
    yaxis_title='Slowdown Factor',
    height=650,
    width=1300,
    hovermode='closest'
)
fig1.update_yaxes(type='log', showgrid=True)
fig1.show()

# =============================================================================
# Figure 2: Token vs Slowdown (colored by expert count)
# =============================================================================
print("Creating Figure 2: Token vs Slowdown...")

fig2 = go.Figure()

hover = [create_hover(row) for _, row in df.iterrows()]
fig2.add_trace(go.Scatter(
    x=df['token'],
    y=df['slowdown_factor'],
    mode='markers',
    marker=dict(
        size=10,
        color=df['expert'],
        colorscale='Viridis',
        colorbar=dict(title='Expert Count'),
        line=dict(width=1, color='black')
    ),
    hovertext=hover,
    hoverinfo='text'
))

fig2.update_layout(
    title='Token Count vs Slowdown Factor',
    xaxis_title='Token Count',
    yaxis_title='Slowdown Factor',
    height=600,
    width=1200,
    hovermode='closest'
)
fig2.update_xaxes(type='log', showgrid=True)
fig2.update_yaxes(type='log', showgrid=True)
fig2.show()

# =============================================================================
# Figure 3: Token vs Expert (colored by slowdown)
# =============================================================================
print("Creating Figure 3: Token vs Expert...")

fig3 = go.Figure()

fig3.add_trace(go.Scatter(
    x=df['token'],
    y=df['expert'],
    mode='markers',
    marker=dict(
        size=12,
        color=df['slowdown_factor'],
        colorscale='RdYlGn_r',
        cmin=1, cmax=10,
        colorbar=dict(title='Slowdown Factor'),
        line=dict(width=1, color='black')
    ),
    hovertext=hover,
    hoverinfo='text'
))

fig3.update_layout(
    title='Token vs Expert Configuration Space<br><sub>Color shows slowdown factor</sub>',
    xaxis_title='Token Count',
    yaxis_title='Expert Count',
    height=600,
    width=1200,
    hovermode='closest'
)
fig3.update_xaxes(type='log', showgrid=True)
fig3.update_yaxes(type='log', showgrid=True)
fig3.show()

# =============================================================================
# Figure 4: Expert vs Model Dim (colored by slowdown)
# =============================================================================
print("Creating Figure 4: Expert vs Model Dim...")

fig4 = go.Figure()

fig4.add_trace(go.Scatter(
    x=df['expert'],
    y=df['model_dim'],
    mode='markers',
    marker=dict(
        size=12,
        color=df['slowdown_factor'],
        colorscale='RdYlGn_r',
        cmin=1, cmax=10,
        colorbar=dict(title='Slowdown Factor'),
        line=dict(width=1, color='black')
    ),
    hovertext=hover,
    hoverinfo='text'
))

fig4.update_layout(
    title='Expert vs Model Dimension Configuration Space<br><sub>Color shows slowdown factor</sub>',
    xaxis_title='Expert Count',
    yaxis_title='Model Dimension',
    height=600,
    width=1200,
    hovermode='closest'
)
fig4.update_xaxes(type='log', showgrid=True)
fig4.update_yaxes(showgrid=True)
fig4.show()

print("\n✅ Created 4 separate interactive plots:")
print("  1. Config progression (colored by kernel type)")
print("  2. Scaling with tokens (colored by expert count)")
print("  3. Token vs Expert space (slowdown as color)")
print("  4. Expert vs Model Dim (slowdown as color)")


Loading best_kernels_with_reference.csv...
Loaded 679 configurations

Creating Figure 1: Config Index vs Slowdown...


Creating Figure 2: Token vs Slowdown...


Creating Figure 3: Token vs Expert...


Creating Figure 4: Expert vs Model Dim...



✅ Created 4 separate interactive plots:
  1. Config progression (colored by kernel type)
  2. Scaling with tokens (colored by expert count)
  3. Token vs Expert space (slowdown as color)
  4. Expert vs Model Dim (slowdown as color)
